# 第6章: 予知（段階的思考）

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行してAPI keyを読み込み、`get_completion`ヘルパー関数を設定します。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ],
        system=system
    )
    return message.content[0].text

---

## レッスン

もし誰かがあなたを起こして、すぐにいくつかの複雑な質問をし、すぐに答えなければならないとしたら、どうでしょうか？**最初に答えを考える時間を与えられた場合**よりも、おそらくうまくいかないでしょう。

なんと、Claudeも同じです。

**Claudeに段階的に考える時間を与えることで、特に複雑なタスクにおいて、Claudeがより正確になることがあります**。ただし、**思考は声に出した場合のみカウントされます**。Claudeに考えるよう求めて答えだけを出力させることはできません - この場合、実際には思考は行われていません。

### 例

以下のプロンプトでは、二番目の文が最初の文と矛盾していることは人間の読者には明らかです。しかし、**Claudeは「関係ない」という言葉をあまりにも字面通りに受け取ってしまいます**。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

Claudeの応答を改善するために、**Claudeが答える前に最初に物事を考え抜くことを許可しましょう**。これは、Claudeがタスクを処理し、考え抜くために取るべき手順を文字通り詳細に説明することで行います。役割プロンプトと組み合わせることで、Claudeがレビューをより深く理解できるようになります。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claudeは順序に敏感な場合があります**。この例は、Claudeが微妙なテキストを理解する能力の最前線にあり、前の例から引数の順序を入れ替えて、ネガティブを最初に、ポジティブを2番目にすると、Claudeの全体的な評価がポジティブに変わります。

ほとんどの状況で（しかし、すべてではありません）、**Claudeは2つの選択肢のうち2番目を選ぶ可能性が高い**です。これは、Webからのトレーニングデータで2番目の選択肢が正しい可能性が高かったからかもしれません。

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**Claudeに考えさせることで、Claudeの答えを間違いから正解に変えることができます**。Claudeが間違いを犯す多くの場合において、これほど簡単なのです！

Claudeの答えが間違っている例を通して、Claudeに考えさせることでそれを修正できる方法を見てみましょう。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

今度は`<brainstorm>`タグでClaudeに段階的に考えさせることでこれを修正しましょう。

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

上記の内容を変更せずにレッスンプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 6.1 - メール分類](#exercise-61---classifying-emails)
- [演習 6.2 - メール分類フォーマット](#exercise-62---email-classification-formatting)

### 演習 6.1 - メール分類
この演習では、Claudeにメールを以下のカテゴリに分類するよう指示します：
- (A) 購入前の質問
- (B) 壊れた、または欠陥のある商品
- (C) 請求に関する質問
- (D) その他（説明してください）

演習の最初の部分では、`PROMPT`を変更して**Claudeが正しい分類だけ、分類のみを出力する**ようにしてください。あなたの答えには**正しい選択肢の文字（A - D）を括弧付きで、さらにカテゴリの名前も含める**必要があります。

各メールがどのカテゴリに分類されるべきかを知るために、`EMAILS`リスト内の各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_1_hint)

まだ行き詰まっていますか？例の解決策については以下のセルを実行してください。

In [ ]:
print(hints.exercise_6_1_solution)

### 演習 6.2 - メール分類フォーマット
この演習では、上記のプロンプトの出力を、まさに私たちが望む通りにフォーマットされた答えを得るように改良します。

お気に入りの出力フォーマット技術を使って、Claudeが正しい分類の文字だけを`<answer></answer>`タグで囲むようにしてください。例えば、最初のメールの答えには正確に`<answer>B</answer>`という文字列が含まれている必要があります。

各メールがどの文字カテゴリが正しいかを忘れた場合は、`EMAILS`リスト内の各メールの横にあるコメントを参照してください。

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_6_2_hint)

### おめでとうございます！

この時点まですべての演習を解決した場合、次の章に進む準備ができています。楽しいプロンプティングを！

---

## サンプル実験場

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響を与えるかを確認するエリアです。

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))